# Standard photometry

In the [previous section](photutils.ipynb), astrometry and photometry was obtained, but the photometry is not in standard and has to be transformed to standard magnitudes. The first step is to import the required packages and read the catalog

In [ ]:
import os.path

from astropy import table
import astropy.coordinates as coord

from astroquery.vizier import Vizier
Vizier.ROW_LIMIT = -1

catalog_name = os.path.join('..', 'resources', 'red_data',
                            'instrumental_photometry.txt')
catalog = table.Table.read(catalog_name, format='ascii.fixed_width')

The transformation from instrumental magnitudes to standard magnitudes requires using two photometric bands to correct any possible color dependency. Therefore, the first step is to select two of the images, obtained with different filters. In the current exercise, the first two images images of each filter are going to be used.

In [ ]:
# Select the first two images of each band

b_image = '../resources/red_data/bfa_m67_b_1.fit'
y_image = '../resources/red_data/bfa_m67_y_1.fit'

b_catalog = catalog[catalog['Filename'] == b_image]
y_catalog = catalog[catalog['Filename'] == y_image]

b_catalog[:5].show_in_notebook()

The two catalog have been read, but now stars in both catalogs have to be identified. Position obtained as part of the astrometry procedure will be used:

In [ ]:
b_pos = coord.SkyCoord(b_catalog['ra'], b_catalog['dec'],
                       unit=('degree', 'degree'))
y_pos = coord.SkyCoord(y_catalog['ra'], y_catalog['dec'],
                       unit=('degree', 'degree'))

pos_by = table.Table(b_pos.match_to_catalog_sky(y_pos), masked=True,
                     names=['index', 'distance', '3d'])
del pos_by['3d']

pos_by[:5].show_in_notebook()

The obtained table identifies the index of the stars in one catalog with the indexes of the same stars in the other catalog. Now, the content of both catalogs can be merged, but the previous commands do not avoid than two stars of one catalog are matched to the same star of another catalog. Therefore, we will use a custom-made function to selects only the nearest star among the possible stars. In addition, the matched stars have to be less than 3 arc seconds appart. 

In [ ]:
def join_catalogs(index_table, left_table, right_table):

    matched = table.hstack([index_table, left_table,
                            right_table[index_table['index']]])
    matched.sort(['index', 'distance'])

    index = matched["index"]
    good_match = [True]
    good_match += [star != oldstar for star, oldstar in zip(index, index[1:])]
    nearby = matched['distance'] < coord.Angle('3 arcsec').degree

    result = matched[good_match * nearby]
    del result['index', 'distance']

    return result

phot_ins = join_catalogs(pos_by, b_catalog, y_catalog)

# Tidy up the resulting column names
for col in phot_ins.colnames:
    new_name = col.replace('_2', '_b').replace('_3','_y')
    phot_ins.rename_column(col, new_name)

# And compute the color
phot_ins['mag_b-y'] = phot_ins['mag_b'] - phot_ins['mag_y']

phot_ins[:5].show_in_notebook()

Now we have all the instrumental information required, but the reference stars with standard magnitudes is missing. For that, the catalog from [Balaguer-Núñez, Galadí-Enríquez, Jordi, C.; 2007](https://ui.adsabs.harvard.edu/abs/2007A%26A...470..585B/abstract) is going to be used. The first step is to retrieve the catalog and match the positions of its stars with the instrumental magnitudes.

In [ ]:
phot_std = Vizier.get_catalogs(['J/A+A/470/585'])[0]
phot_std = phot_std[~phot_std['b-y'].mask]  # Reject stars with missing b-y

pos_std = coord.SkyCoord(phot_std['RAJ2000'], phot_std['DEJ2000'],
                         unit=('hourangle', 'degree'))
pos_ins = coord.SkyCoord(phot_ins['ra_b'], phot_ins['dec_b'], 
                         unit=('degree', 'degree'))

pos_std_ins = table.Table(pos_std.match_to_catalog_sky(pos_ins),
                          masked=True, names=['index', 'distance', '3d'])
del pos_std_ins['3d']

phot_std_ins = join_catalogs(pos_std_ins, phot_std, phot_ins)

print('Number of matched stars:', len(phot_std_ins), 'of', len(phot_ins))
phot_std_ins[:5].show_in_notebook()

The final step is to find the transformation between instrumental and standard magnitudes. In this case, the equation to be solved is:

$$(b-y)_{std}=m\cdot(b-y)_{ins}+b$$

where the coefficients $m$ and $b$ have to be found.

In [ ]:
%matplotlib widget
import matplotlib.pyplot as plt

# It would be better to use scipy.optimize.least_squares
from scipy.stats import linregress 

regression = stats.linregress(phot_std_ins['mag_b-y'], phot_std_ins['b-y'])

fit = regression.intercept + regression.slope * phot_std_ins['mag_b-y']
phot_std_ins['final_b-y'] = fit

print('Slope:', regression.slope)
print('Zeropoint:', regression.intercept)

plt.plot(phot_std_ins['mag_b-y'], phot_std_ins['b-y'], 'o')
plt.plot(phot_std_ins['mag_b-y'], fit, 'r')
plt.xlabel('$(b-y)_{ins}$')
plt.ylabel('$(b-y)_{std}$')

phot_std_ins['ra_b', 'dec_b', 'final_b-y'][:5]